# Comparing Inference Latency of Models Served with Flask and FastAPI on SageMaker Endpoints

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook.

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

---

## Background

This notebook will demontrate how the inference latency of the same model varies when one model is served with [Python Flask](https://flask.palletsprojects.com/en/2.3.x/), [FastAPI](https://fastapi.tiangolo.com/) and [SageMaker pre-built container](https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html). In all cases, we use a [decision tree](https://scikit-learn.org/stable/modules/tree.html) algorithm from [scikit-learn](https://scikit-learn.org/stable/) package which is trained on the famous [iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set). 

The scripts and this notebook can be used to load test and compare latency for any endpoint that you deploy on SageMaker. You can either test a [pre-built SageMaker container](https://docs.aws.amazon.com/sagemaker/latest/dg/docker-containers-prebuilt.html) or [bring your own container](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-byoc-containers.html). [This GitHub repo](https://github.com/aws/amazon-sagemaker-examples/tree/main/advanced_functionality/scikit_bring_your_own) provides a more detailed example on how to bring your own container to SageMaker

## Permissions

Running this notebook requires `SageMakerFullAccess` permissions. Additionally you will need permissions to publish to Amazon ECR. You can achieve this by adding the managed policy `AmazonEC2ContainerRegistryFullAccess` to the role that you used to start your notebook instance. There's no need to restart your notebook instance when you do this, the new permissions will be available immediately.

## The experiment

Load testing is carried out on `ml.c5d.18xlarge` SageMaker Notebook Instance type. It is recommended to use a large instance type for load testing in order to be able to provide the adeqate load on the Inference Endpoints

# Part 1: Bundling and Pushing Models to Amazon ECR

There is a script inside each model folder to build and push a model image to ECR. The script logs in to ECR, builds and tags an image and pushes the image to ECR

In [ ]:
models = ["flask-model", "fastapi-model"]

In [ ]:
import subprocess

for model in models:
    try:
        result = subprocess.run(["./build_and_push.sh", f"{model}"],
                                capture_output=True, text=True)
        if result.returncode == 0:
            print(f"Successfully built and pushed {model} to Amazon ECR")
        else:
            print(f"Error building and pushing {model}:", result.stderr)
    except Exception as e:
        print("Exception:", str(e))

# Part 2: Training and Deploying models to SageMaker Inference Endpoints

In [ ]:
import boto3
import re
import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
import sagemaker as sage
from time import gmtime, strftime
from sagemaker.serializers import CSVSerializer

# S3 prefix
prefix = "sagemaker-endpoint-testing-example"

# Define IAM role
role = get_execution_role()
sess = sage.Session()
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name

In [ ]:
# Define data location
WORK_DIRECTORY = "data"
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [ ]:
# Train and deploy each model
for model in models:
    image = f"{account}.dkr.ecr.{region}.amazonaws.com/{model}:latest"

    tree = sage.estimator.Estimator(
        image,
        role,
        1,
        "ml.c4.2xlarge",
        output_path=f"s3://{sess.default_bucket()}/output",
        sagemaker_session=sess,
    )

    tree.fit(data_location)

    tree.deploy(1, "ml.m4.xlarge", serializer=CSVSerializer(),
                endpoint_name=model)

# Part 3: Deploying SageMaker Pre-built Scikit-learn container

In [ ]:
from sklearn.model_selection import train_test_split
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.serializers import CSVSerializer

In [ ]:
%%writefile script.py

#!/usr/bin/env python

# A sample training component that trains a simple scikit-learn decision tree model.
# This implementation works in File mode and makes no assumptions about the input file names.
# Input is specified as CSV with a data point in each row and the labels in the first column.

from __future__ import print_function

import json
import os
import pickle
import sys
import traceback
import pickle


import pandas as pd
from sklearn import tree

# These are the paths to where SageMaker mounts interesting things in your container.

prefix = '/opt/ml/'

input_path = prefix + 'input/data'
output_path = os.path.join(prefix, 'output')
model_path = os.path.join(prefix, 'model')
param_path = os.path.join(prefix, 'input/config/hyperparameters.json')

# This algorithm has a single channel of input data called 'training'. Since we run in
# File mode, the input files are copied to the directory specified here.
channel_name='training'
training_path = os.path.join(input_path, channel_name)


def model_fn(model_dir):
    #Load Model
    with open(os.path.join(model_dir, "decision-tree-model.pkl"), "rb") as inp:
        clf = pickle.load(inp)
    return clf


# The function to execute the training.
def train():
    print('Starting the training.')
    try:
        # Read in any hyperparameters that the user passed with the training job
        with open(param_path, 'r') as tc:
            trainingParams = json.load(tc)

        # Take the set of files and read them all into a single pandas dataframe
        input_files = [ os.path.join(training_path, file) for file in os.listdir(training_path) ]
        if len(input_files) == 0:
            raise ValueError(('There are no files in {}.\n' +
                              'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                              'the data specification in S3 was incorrectly specified or the role specified\n' +
                              'does not have permission to access the data.').format(training_path, channel_name))
        raw_data = [ pd.read_csv(file, header=None) for file in input_files if file.endswith(".csv")]
        train_data = pd.concat(raw_data)

        # labels are in the first column
        train_y = train_data.iloc[:,0]
        train_X = train_data.iloc[:,1:]

        # Here we only support a single hyperparameter. Note that hyperparameters are always passed in as
        # strings, so we need to do any necessary conversions.
        max_leaf_nodes = trainingParams.get('max_leaf_nodes', None)
        if max_leaf_nodes is not None:
            max_leaf_nodes = int(max_leaf_nodes)

        # Now use scikit-learn's decision tree classifier to train the model.
        clf = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
        clf = clf.fit(train_X, train_y)
        
        model_dir = os.environ['SM_MODEL_DIR']

        # save the model
        with open(os.path.join(model_dir, 'decision-tree-model.pkl'), 'wb') as out:
            pickle.dump(clf, out)
        print('Training complete.')
    except Exception as e:
        # Write out an error file. This will be returned as the failureReason in the
        # DescribeTrainingJob result.
        trc = traceback.format_exc()
        with open(os.path.join(model_dir, 'failure'), 'w') as s:
            s.write('Exception during training: ' + str(e) + '\n' + trc)
        # Printing this causes the exception to be in the training job logs, as well.
        print('Exception during training: ' + str(e) + '\n' + trc, file=sys.stderr)
        # A non-zero exit code causes the training job to be marked as Failed.
        sys.exit(255)

if __name__ == '__main__':
    train()

    # A zero exit code causes the job to be marked a Succeeded.
    sys.exit(0)

In [ ]:
pre_built_enpoint_name = 'sagemaker-scikit-pre-built'

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.c5.xlarge",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="sagemaker-scikit-tree",
)

In [ ]:
sklearn_estimator.fit(data_location)

In [ ]:
predictor = sklearn_estimator.deploy(1, "ml.m4.xlarge", serializer=CSVSerializer(),
                endpoint_name=pre_built_enpoint_name)

# Adding new endpoint to our models list
models.append(pre_built_enpoint_name)

# Part 4: Load Testing and Measuring Latency of Each Endpoint

In [ ]:
%%capture output

!pip install locust

if output.stderr:
    print("An error occurred:", output.stderr)

## Launch

A test for each endpoint takes around 5 minutes to run

In [ ]:
import subprocess
for model in models:
    try:
        result = subprocess.run(["./load-testing/run_locust.sh", f"{model}"],
                                capture_output=True, text=True)
        if result.returncode == 0:
            print(f"Finished load testing for {model} endpoint")
        else:
            print(f"Error load testing {model} endpoint:", result.stderr)
    except Exception as e:
        print("Exception:", str(e))

# Part 5: Visualising the results

In [ ]:
latency_dict = {}

# for each endpoint get latency from the load test results
for model in models:
    df = pd.read_csv(f"./load-testing/results_{model}_stats.csv")

    min_latency = round(df.tail(1)['Min Response Time'].values[0] / 1000, 1)
    avg_latency = round(df.tail(1)['Average Response Time'].values[0] / 1000, 1)
    max_latency = round(df.tail(1)['Max Response Time'].values[0] / 1000, 1)

    latency_dict.update({
        model: [min_latency, avg_latency, max_latency]
    })

# generate a data frame of the results and plot a box plot
results = pd.DataFrame(latency_dict)
res_plot = results.plot(legend=True, figsize=(12, 7), kind='box',
            title="Request Latency Across Model Types",
            xlabel="Model Type",
            ylabel="Latency in Seconds")

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.


![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/sagemaker_endpoints_latency_testing|endpoint_testing.ipynb)
